In [6]:
from preprocessing import get_LSTM_dfs, get_features_and_target
import tensorflow as tf

In [9]:
df = get_features_and_target(
    "BTC/USDT", day_to_forecast=1, feature_lags=[3, 9, 16, 24], model="LSTM"
)
X = df.drop(columns=f"BTC:USDT_target")

train_df, valid_df = get_LSTM_dfs()

In [14]:
conv1 = tf.keras.layers.Conv1D(
    filters=60,
    kernel_size=5,
    strides=1,
    padding="causal",
    activation="relu",
    input_shape=[None, len(X.columns)],
)

# Bidirectional LSTM tf.keras.layers
lstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(30, return_sequences=True))
lstm2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True))

# Model construction
inputs = tf.keras.layers.Input(shape=(None, len(X.columns)))
x = conv1(inputs)
x = lstm1(x)
x = lstm2(x)
x = tf.keras.layers.Dense(60, activation="relu")(x)
x = tf.keras.layers.Dropout(0.25)(x)
outputs = tf.keras.layers.Dense(1)(x)

# SGD optimizer and Huber loss
optimizer = tf.keras.optimizers.legacy.SGD(
    learning_rate=1e-5, momentum=0.9, nesterov=True
)
loss = tf.keras.losses.Huber()

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer, loss, metrics=["mae"])
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None, 79)]        0         
                                                                 
 conv1d_4 (Conv1D)           (None, None, 60)          23760     
                                                                 
 bidirectional_8 (Bidirecti  (None, None, 60)          21840     
 onal)                                                           
                                                                 
 bidirectional_9 (Bidirecti  (None, None, 40)          12960     
 onal)                                                           
                                                                 
 dense_8 (Dense)             (None, None, 60)          2460      
                                                                 
 dropout_4 (Dropout)         (None, None, 60)          0   

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
tensorboard = tf.keras.callbacks.TensorBoard("tensorboard_logs")

model.fit(
    train_df,
    validation_data=valid_df,
    callbacks=[early_stopping, tensorboard],
    epochs=50,
)

